In [ ]:
! pip install bs4


     |██▉                             | 10 kB 32.3 MB/s eta 0:00:01
     |█████▋                          | 20 kB 35.0 MB/s eta 0:00:01
     |████████▌                       | 30 kB 41.6 MB/s eta 0:00:01
     |███████████▎                    | 40 kB 34.4 MB/s eta 0:00:01
     |██████████████▏                 | 51 kB 18.8 MB/s eta 0:00:01
     |█████████████████               | 61 kB 17.5 MB/s eta 0:00:01
     |███████████████████▉            | 71 kB 18.2 MB/s eta 0:00:01
     |██████████████████████▋         | 81 kB 16.9 MB/s eta 0:00:01
     |█████████████████████████▌      | 92 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████▎   | 102 kB 16.2 MB/s eta 0:00:01
     |███████████████████████████████▏| 112 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 16.2 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=59a098135504c24bec5f47edc69a36ecff9a0660630fdaea6df0ad11da192a25
  Stored in directory: /home/jovyan/.cache/pip/w

In [ ]:
! pip install tqdm


     |████▋                           | 10 kB 22.9 MB/s eta 0:00:01
     |█████████▎                      | 20 kB 27.3 MB/s eta 0:00:01
     |█████████████▉                  | 30 kB 29.5 MB/s eta 0:00:01
     |██████████████████▌             | 40 kB 33.5 MB/s eta 0:00:01
     |███████████████████████         | 51 kB 21.0 MB/s eta 0:00:01
     |███████████████████████████▊    | 61 kB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 10.3 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import bs4
import traceback
import json
import pandas as pd
from requests.exceptions import RequestException
import time
import sys
import random
from tqdm import tqdm

In [ ]:
f = open('agg_data.json',) 

data_dict = json.load(f) 

f.close()

In [ ]:
def getVideoInfo(bvid, aid):
    video_url = "https://www.bilibili.com/video/" + bvid
    videoInfoDict = {}
    
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36'}
    response=requests.get(url=video_url,headers=headers,timeout=20)
    response.raise_for_status()
    response.encoding=response.apparent_encoding
    html = response.text
    soup=BeautifulSoup(html,'html.parser')
    
    #get cats of the video
    catList = []
    if(soup.find("span", "a-crumbs")!=None):
        for a in soup.find("span", "a-crumbs"):
            if (a.name == "a"):
                catList.append(a.text)
        videoInfoDict["cats"] = catList
        #get the publish time
        videoInfoDict["pubtime"] = soup.find("span", "a-crumbs").next_sibling.text
    else:
        videoInfoDict["cats"] = {}
        videoInfoDict["pubtime"] = "none"
    
    
    #get the tags
    tagList = []
    if(soup.find("ul", "tag-area")!=None):
        for tag in soup.find("ul", "tag-area").children:
            if isinstance(tag, bs4.element.Tag):
                tagList.append(tag.text)
        videoInfoDict["tags"] = tagList
    else:
        videoInfoDict["tags"] = []
    
    video_info_url = "https://api.bilibili.com/x/web-interface/view?aid="+aid
    response1 =requests.get(url=video_info_url,timeout=20)
    response1.encoding=response1.apparent_encoding
    html = response1.text
    if response1.status_code == 200:
        j1 = json.loads(html)
        if("data" in j1):
            j1 = j1["data"]
            videoInfoDict['title'] = j1['title']
            videoInfoDict['view'] = j1['stat']['view']  # 播放量,视频没有播放量时显示会‘--’,不是整数，会抓取失败
            videoInfoDict['danmaku'] = j1['stat']['danmaku']  # 弹幕
            videoInfoDict['favorite'] = j1['stat']['favorite']  # 收藏
            videoInfoDict['coin'] = j1['stat']['coin']  # 硬币
            videoInfoDict['share'] = j1['stat']['share']  # 分享
            videoInfoDict['reply'] = j1['stat']['reply']  # 评论
            videoInfoDict['like'] = j1['stat']['share']  # 点赞
            videoInfoDict['duration'] = j1['duration']  # 时长
            videoInfoDict['cid'] = j1['cid']  # cid
            videoInfoDict['aid'] = j1['aid']  # aid
            videoInfoDict['desc'] = j1['desc']  # aid
        else:
            print("can't find video info of "+video_url)
            videoInfoDict['title'] = "none"
            videoInfoDict['view'] = "none"
            videoInfoDict['danmaku'] = "none"
            videoInfoDict['favorite'] = "none"
            videoInfoDict['coin'] = "none"
            videoInfoDict['share'] = "none"
            videoInfoDict['reply'] = "none"
            videoInfoDict['like'] = "none"
            videoInfoDict['duration'] = "none"
            videoInfoDict['cid'] = "none"
            videoInfoDict['aid'] = "none"
            videoInfoDict['desc'] = "none"
    
    return videoInfoDict

In [ ]:
res1 = dict(list(data_dict.items())[:50]) 
res2 = dict(list(data_dict.items())[50:100])
res3 = dict(list(data_dict.items())[100:150])
res4 = dict(list(data_dict.items())[150:200])
res5 = dict(list(data_dict.items())[200:250])
res6 = dict(list(data_dict.items())[250:])
len(res1.keys()), len(res2.keys()), len(res3.keys()), len(res4.keys()), len(res5.keys()), len(res6.keys())

(50, 50, 50, 50, 50, 42)

In [ ]:
video_dict1 = {}
video_dict2 = {}
video_dict3 = {}
video_dict4 = {}
video_dict5 = {}
video_dict6 = {}

In [ ]:
data_dict_sub = res1
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict1[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict1.json', 'w') as json_file:
    json.dump(video_dict1, json_file)


 76%|███████▌  | 38/50 [2:15:18<38:54, 194.54s/it]can't find video info of https://www.bilibili.com/video/BV144411k7NN
can't find video info of https://www.bilibili.com/video/BV1Lt41157Cc
can't find video info of https://www.bilibili.com/video/BV13x411Z7NU
can't find video info of https://www.bilibili.com/video/BV1o4411V7sf
can't find video info of https://www.bilibili.com/video/BV1W4411W78w
can't find video info of https://www.bilibili.com/video/BV1P4411j7J5
can't find video info of https://www.bilibili.com/video/BV1t4411Y784
can't find video info of https://www.bilibili.com/video/BV1g4411v7Y5

 80%|████████  | 40/50 [2:22:34<34:44, 208.46s/it]


100%|██████████| 50/50 [2:53:56<00:00, 208.73s/it]


In [ ]:
data_dict_sub = res2
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict2[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict2.json', 'w') as json_file:
    json.dump(video_dict2, json_file)


100%|██████████| 50/50 [2:36:48<00:00, 188.16s/it]


In [ ]:
data_dict_sub = res3
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict3[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict3.json', 'w') as json_file:
    json.dump(video_dict3, json_file)


100%|██████████| 50/50 [2:32:41<00:00, 183.23s/it]


In [ ]:
data_dict_sub = res4
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict4[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict4.json', 'w') as json_file:
    json.dump(video_dict4, json_file)


100%|██████████| 50/50 [2:20:59<00:00, 169.19s/it]


In [ ]:
data_dict_sub = res5
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict5[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict5.json', 'w') as json_file:
    json.dump(video_dict5, json_file)


100%|██████████| 50/50 [2:22:13<00:00, 170.67s/it]


In [ ]:
data_dict_sub = res6
for author_url in tqdm(data_dict_sub):
    video_list = data_dict_sub[author_url]['video_list']
    for video in video_list:
        video_dict6[video['bvid']] = getVideoInfo(video['bvid'], str(video['aid']))

with open('video_dict6.json', 'w') as json_file:
    json.dump(video_dict6, json_file)


100%|██████████| 42/42 [1:50:20<00:00, 157.64s/it]


In [ ]:
def merge_dicts(*dict_args):
    """
    Given any number of dictionaries, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dictionaries.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

In [ ]:
with open('video_dict1.json', 'r') as json_file:
    video_dict1 = json.load(json_file)
with open('video_dict2.json', 'r') as json_file:
    video_dict2 = json.load(json_file)
with open('video_dict3.json', 'r') as json_file:
    video_dict3 = json.load(json_file)
with open('video_dict4.json', 'r') as json_file:
    video_dict4 = json.load(json_file)
with open('video_dict5.json', 'r') as json_file:
    video_dict5 = json.load(json_file)
with open('video_dict6.json', 'r') as json_file:
    video_dict6 = json.load(json_file)

agg_dict = merge_dicts(video_dict1, video_dict2, video_dict3, video_dict4, video_dict5, video_dict6)

In [ ]:
import pandas as pd
data = pd.read_csv("clean_data.csv")

In [ ]:
data.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace=True)
data.head()

,aid,author,bvid,created,description,comment,follower,length,likes,play,title,typeid,video_review,top1,top1_count,top2,top2_count,top3,top3_count
0,797419793.0,拜托了小翔哥,BV1qy4y1C7fS,2020-10-15 17:53:29,记得戴上耳机，深夜躲在被窝里二刷，所有的不开心都会被治愈！,2188,2840710,396,4995407,1040090,小伙网上学来的饺子馅秘方，晚上肚子饿，起来在床边偷吃了一大碗,美食圈,7498,生活,17.0,美食,14.0,none,0.0
1,627391341.0,拜托了小翔哥,BV1rt4y1v7ZK,2020-10-13 17:49:18,小伙一天做了三盆口味鸡爪，最好吃的原来是这一款,1589,2840710,422,4995407,995238,小伙一天做了三盆口味的鸡爪，只为找到最好吃的那一款,美食圈,3797,生活,17.0,美食,14.0,none,0.0
2,372483730.0,拜托了小翔哥,BV18Z4y1L74x,2020-10-12 19:06:54,网传蒙上眼睛堵住鼻子会分不出可乐和雪碧，小伙不信试了一下，结果却大吃一惊。,1842,2840710,326,4995407,1015514,听说蒙上眼睛堵住鼻子会分不出可乐和雪碧，试一试,搞笑,2905,生活,17.0,美食,14.0,none,0.0
3,287261155.0,拜托了小翔哥,BV1Rf4y1D7az,2020-10-08 18:29:21,风靡世界的欧洲版沙县小吃，小伙在家大口吃滋滋冒油的土耳其烤肉,2096,2840710,406,4995407,1514605,398买的烤炉自制mini版土耳其烤肉，大肉片滋滋冒油太香啦,美食圈,6673,生活,17.0,美食,14.0,none,0.0
4,202373061.0,拜托了小翔哥,BV18h41197fY,2020-10-05 17:59:08,爷青回，印度小吃应该是第四集了吧，大家还喜欢看的话，我又继续给大家拍。,2835,2840710,397,4995407,1371182,跟印度小哥学做咖喱，原来是这样做的，以后再也不用去超市买了,美食侦探,9157,生活,17.0,美食,14.0,none,0.0


In [ ]:
agg_dict['BV1qy4y1C7fS'].keys()

dict_keys(['cats', 'pubtime', 'tags', 'title', 'view', 'danmaku', 'favorite', 'coin', 'share', 'reply', 'like', 'duration', 'cid', 'aid', 'desc'])

In [ ]:
data[data["bvid"]=='BV1qy4y1C7fS']

,aid,author,bvid,created,description,comment,follower,length,likes,play,title,typeid,video_review,top1,top1_count,top2,top2_count,top3,top3_count
0,797419793.0,拜托了小翔哥,BV1qy4y1C7fS,2020-10-15 17:53:29,记得戴上耳机，深夜躲在被窝里二刷，所有的不开心都会被治愈！,2188,2840710,396,4995407,1040090,小伙网上学来的饺子馅秘方，晚上肚子饿，起来在床边偷吃了一大碗,美食圈,7498,生活,17.0,美食,14.0,none,0.0


In [ ]:
max_len = 0
for bvid in agg_dict:
    cur_len = len(agg_dict[bvid]['tags'])
    if cur_len>max_len:
        max_len = cur_len
max_len

19

In [ ]:
data_df = pd.DataFrame()
list_tags = []
for bvid in agg_dict:
    video_dict = agg_dict[bvid]
    list_tags.append(agg_dict[bvid]["tags"])

maxLen = max(map(len, list_tags))
list_tags_1 = [row.extend(['none']*(maxLen - len(row))) for row in list_tags]

In [ ]:
video_df = pd.DataFrame(data=list_tags, index = agg_dict.keys(), columns=["tag1","tag2","tag3","tag4"\
,"tag5","tag6","tag7","tag8","tag9","tag10","tag11","tag12","tag13","tag14","tag15","tag16","tag17","tag18","tag19",])
video_df.head()

,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag10,tag11,tag12,tag13,tag14,tag15,tag16,tag17,tag18,tag19
BV1qy4y1C7fS,深夜食堂,家常菜,美食,美食,美食制作,\n B站美食大侦探悬赏令\n,\n 美食vlog\n,\n 料理制作\n,\n 饺子\n,\n 包饺子\n,\n 宵夜\n,,none,none,none,none,none,none,none
BV1rt4y1v7ZK,美食,美食,美食制作,\n 美食vlog\n,\n 料理制作\n,\n 卤鸡爪\n,\n 泡椒鸡爪\n,\n 虎皮鸡爪\n,,none,none,none,none,none,none,none,none,none,none
BV18Z4y1L74x,搞笑,生活,搞笑,\n 全能沙雕\n,\n 沙雕之主\n,\n 逗比\n,\n 生活\n,\n 魔性\n,\n 挑战\n,\n 沙雕\n,,none,none,none,none,none,none,none,none
BV1Rf4y1D7az,美食,美食,美食制作,\n 十八般厨艺大赏\n,\n 料理制作\n,\n 美食vlog\n,\n 街边摊小吃\n,\n 土耳其烤肉\n,,none,none,none,none,none,none,none,none,none,none
BV18h41197fY,印度美食,街头美食,美食,美食,美食侦探,\n B站美食大侦探悬赏令\n,\n 咖喱\n,\n 印度小吃\n,\n 咖喱饭\n,\n 印度咖喱\n,\n 自制咖喱\n,,none,none,none,none,none,none,none


In [ ]:
data.set_index(["bvid"],inplace=True)

In [ ]:
data_all = pd.concat([data, video_df], axis=1, join='inner')
data_all.to_csv("data_all.csv")

In [ ]:
data_all

,aid,author,created,description,comment,follower,length,likes,play,title,...,tag10,tag11,tag12,tag13,tag14,tag15,tag16,tag17,tag18,tag19
BV1qy4y1C7fS,797419793.0,拜托了小翔哥,2020-10-15 17:53:29,记得戴上耳机，深夜躲在被窝里二刷，所有的不开心都会被治愈！,2188,2840710,396,4995407,1040090,小伙网上学来的饺子馅秘方，晚上肚子饿，起来在床边偷吃了一大碗,...,\n 包饺子\n,\n 宵夜\n,,none,none,none,none,none,none,none
BV1rt4y1v7ZK,627391341.0,拜托了小翔哥,2020-10-13 17:49:18,小伙一天做了三盆口味鸡爪，最好吃的原来是这一款,1589,2840710,422,4995407,995238,小伙一天做了三盆口味的鸡爪，只为找到最好吃的那一款,...,none,none,none,none,none,none,none,none,none,none
BV18Z4y1L74x,372483730.0,拜托了小翔哥,2020-10-12 19:06:54,网传蒙上眼睛堵住鼻子会分不出可乐和雪碧，小伙不信试了一下，结果却大吃一惊。,1842,2840710,326,4995407,1015514,听说蒙上眼睛堵住鼻子会分不出可乐和雪碧，试一试,...,\n 沙雕\n,,none,none,none,none,none,none,none,none
BV1Rf4y1D7az,287261155.0,拜托了小翔哥,2020-10-08 18:29:21,风靡世界的欧洲版沙县小吃，小伙在家大口吃滋滋冒油的土耳其烤肉,2096,2840710,406,4995407,1514605,398买的烤炉自制mini版土耳其烤肉，大肉片滋滋冒油太香啦,...,none,none,none,none,none,none,none,none,none,none
BV18h41197fY,202373061.0,拜托了小翔哥,2020-10-05 17:59:08,爷青回，印度小吃应该是第四集了吧，大家还喜欢看的话，我又继续给大家拍。,2835,2840710,397,4995407,1371182,跟印度小哥学做咖喱，原来是这样做的，以后再也不用去超市买了,...,\n 印度咖喱\n,\n 自制咖喱\n,,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV1ps411e7pP,7535387.0,司徒建国Stu,2016-12-15 13:55:38,你知道吗，你所相信尊敬的外教老师，帮你取的英文名竟然有可能是为了搞笑？以前也做过外教的司徒老...,74,332213,202,1125470,18974,【司徒】告诉你个秘密 别让外教帮你起英文名！,...,\n 司徒\n,\n 外教\n,,none,none,none,none,none,none,none
BV1ns411s7TU,7307790.0,司徒建国Stu,2016-11-29 17:29:35,司徒今天走深情路线，为我们带来英国后朋克和另类摇滚乐队The Kooks的火热单曲Naive...,49,332213,166,1125470,3505,【司徒】深情弹唱英国The Kooks傻瓜乐队Naive,...,\n The Kooks\n,,none,none,none,none,none,none,none,none
BV1es411W73Y,7195777.0,司徒建国Stu,2016-11-21 16:07:58,英国逗比司徒正经起来还是很帅的！ 这次他挑战的是黑人灵魂音乐No Diggity，自弹自唱超...,52,332213,199,1125470,4489,【司徒】英国小哥翻唱黑人灵魂乐No Diggity超有味道,...,\n 灵魂\n,\n 黑人\n,,none,none,none,none,none,none,none
BV1vs411t7Hj,6370441.0,司徒建国Stu,2016-09-22 10:46:39,中文十级的老外【司徒】勇敢挑战高难度中国歌，周杰伦经典中国风歌曲《发如雪》，逗比展现深情一面...,112,332213,147,1125470,20719,【司徒】老外中文弹唱周杰伦经典歌曲发如雪,...,none,none,none,none,none,none,none,none,none,none
